# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute forecast data, respectively.

### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
IS = PowerSystems.IS

InfrastructureSystems

### Data management packages

In [2]:
using Dates
using DataFrames

### Optimization packages

In [3]:
using JuMP
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data. We'll only use the hourly and 5-minute data for the
example simulations below, but the 6-second data is included for future development.

In [4]:
base_dir = PowerSystems.download(PowerSystems.TestData; branch = "master");
pm_data = PowerSystems.PowerModelsData(joinpath(base_dir, "matpower", "case5_re_uc.m"))

FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts", "7day")

tsp_da = IS.read_time_series_metadata(joinpath(
    FORECASTS_DIR,
    "timeseries_pointers_da_7day.json",
))
tsp_rt = IS.read_time_series_metadata(joinpath(
    FORECASTS_DIR,
    "timeseries_pointers_rt_7day.json",
))
tsp_agc = IS.read_time_series_metadata(joinpath(
    FORECASTS_DIR,
    "timeseries_pointers_agc_7day.json",
))

sys_DA = System(pm_data)
reserves = [
    VariableReserve{ReserveUp}("REG1", true, 5.0, 0.1),
    VariableReserve{ReserveUp}("REG2", true, 5.0, 0.06),
    VariableReserve{ReserveUp}("REG3", true, 5.0, 0.03),
    VariableReserve{ReserveUp}("REG4", true, 5.0, 0.02),
]
contributing_devices = get_components(Generator, sys_DA)
for r in reserves
    add_service!(sys_DA, r, contributing_devices)
end

add_forecasts!(sys_DA, tsp_da)

sys_RT = System(pm_data)
add_forecasts!(sys_RT, tsp_rt)

sys_AGC = System(pm_data)
add_forecasts!(sys_AGC, tsp_agc)

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data: gen_name 5x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/matpower.jl:665
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/data.jl:1218
┌ Info: the voltage setpoint on generator 4 does not match the value at bus 4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/data.jl:1686
┌ Info: the voltage setpoint on generator 1 does not match the value at bus 1
└ @ PowerSystems /Users/cbarrows/.julia/packages/Po

## `OperationsProblemTemplate`s

In [5]:
template_uc = template_unit_commitment()
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalDispatch
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = PowerSystems.TapTransformer
        formulation = PowerSimulations.StaticTransformer
      L:
        device_type = PowerSystems.Line
 

### Define the Simulation Sequence

In [6]:
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template_uc, sys_DA, solver),
    "ED" => Stage(GenericOpProblem, template_ed, sys_RT, solver),
)

feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_source_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

cache = Dict("UC" => [TimeStatusChange(ThermalStandard, PSI.ON)])

order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 24, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
    #cache = cache,
)

Feed Forward Chronology
-----------------------

ED: PowerSimulations.SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [7]:
file_path = tempdir()
sim = Simulation(
    name = "5bus-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = file_path,
)

Simulation()


### Build simulation

In [8]:
build!(sim)

### Execute simulation

In [9]:
sim_results = execute!(sim)

Executing Step 1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: May 23 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Presolve 156 (-3144) rows, 720 (-4308) columns and 864 (-7452) elements
Perturbing problem by 0.001% of 1000000 - largest nonzero change 0.10877769 ( 0.008377309%) - largest zero change 0.10433902
0  Obj 7.6670803 Primal inf 202.26429 (156)
78  Obj 18433.08 Primal inf 125.35802 (115)
156  Obj 30751.009 Primal inf 41.639392 (52)
206  Obj 34145.824
Optimal - objective value 34129.959
After Postsolve, objective 34129.959, infeasibilities - dual 53844.549 (275), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 34129.959
Optimal - objective value 34129.959
Optimal objective 34129.95865 - 206 iterations time 0.012, Presolve 0.00
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Welcome to the CBC MILP Solver 
Version: 2.10.3 
B

Simulation Results Reference

Results Folder: /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/5bus-test/3/results

Reference Tables

________________

stage-UC

start__ThermalStandard: dataframe size (1, 3)

P__ThermalStandard: dataframe size (1, 3)

REG2__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

parameter_service_requirement_VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

stop__ThermalStandard: dataframe size (1, 3)

REG4__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

REG1__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

REG3__VariableReserve_PowerSystems.ReserveUp: dataframe size (1, 3)

parameter_P_PowerLoad: dataframe size (1, 3)

On__ThermalStandard: dataframe size (1, 3)

stage-ED

P__ThermalStandard: dataframe size (24, 3)

parameter_P_PowerLoad: dataframe size (24, 3)

stage-UC

time length: 24

stage-ED

time length: 12

## Results

In [10]:
ed_results = load_simulation_results(sim_results, "ED");

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*